In [ ]:
# The aim of this module is to extract data from yahood finance and export to pipe delimitered CSV files

# Exports are:
# R_ALL_TICKER >> list of all tickers in scope
# R_ALL_INFO >> point in time information
# R_ALL_FIN >> financial information, balance sheet, income statement and cashflow
# R_ALL_Prices >> historical prices over last x months
# R_ALL_Div >> dividend price over time


In [1]:
import pandas as pd
import yfinance as yf
import numpy as np


# File name convention
# R_ = raw extract 
# E_ = Entry files into enrichment layer
# C_ = enriched layer with calculation
# I = insights layer, designed for model baselines
# V_ = validation

In [2]:
#ASX Tickers
RAW_TICKER_ASX = pd.read_csv('https://www.asx.com.au/asx/research/ASXListedCompanies.csv',skiprows=1)
RAW_TICKER_ASX["Ticker"] = RAW_TICKER_ASX['ASX code']+".AX"
TICKER_ASX = list(RAW_TICKER_ASX['Ticker'].dropna())

In [5]:
# Nasdaq Tickers 

# Note: need to find new site, the link below no longer works
#ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt
#ftp.nasdaqtrader.com/SymbolDirectory/otherlisted.txt

# RAW_TICKER_NDQ = pd.read_csv('/Users/joezhou/Downloads/nasdaqlisted.txt',sep="|")
# RAW_TICKER_NDQ["Ticker"] = RAW_TICKER_NDQ['Symbol']
# TICKER_NDQ = list(RAW_TICKER_NDQ['Ticker'].dropna())

In [3]:
#London Stock Exchange Tickers
RAW_TICKER_LSE = pd.read_excel (r'https://docs.londonstockexchange.com/sites/default/files/documents/list_of_sets_securities_14.xls', sheet_name='SETS',skiprows=3)
RAW_TICKER_LSE["Ticker"] = RAW_TICKER_LSE['Mnemonic']+".L"
TICKER_LSE = list(RAW_TICKER_LSE['Ticker'].dropna())

In [ ]:
#NYSE Tickers
# RAW_TICKER_NYSE = pd.read_excel (r'https://www.theice.com/publicdocs/data/NYSE_Equity_Index_Ticker_List.xlsx', sheet_name='SETS',skiprows=3)

#Note: Need to find a better data source

In [3]:
#Append list for data extraction layer
TICKER_ALL = TICKER_ASX
# TICKER_ALL = (TICKER_ASX + TICKER_NDQ + TICKER_LSE)

TICKER_ALL.sort()
four_split = np.array_split(TICKER_ALL, 4)

TICKER_ALL_1 = four_split[0]
TICKER_ALL_2 = four_split[1]
TICKER_ALL_3 = four_split[2]
TICKER_ALL_4 = four_split[3]

#Remove Duplicates
# TICKER_ALL = list(dict.fromkeys(TICKER_ALL))


# Export all ticker list for referencing purposes
df_x = pd.DataFrame(TICKER_ALL, columns=["colummn"])
df_x.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_TICKER.pkl")


In [ ]:
#Sample top 5 for testing code purposes
# TICKER_ASXs = TICKER_ASX[:5]
# TICKER_NDQs = TICKER_NDQ[:5]
# TICKER_LSEs = TICKER_LSE[:5]

# TICKER_ALL = (TICKER_ASXs + TICKER_NDQs + TICKER_LSEs)

In [ ]:
#Download part 1: Company Information

INF_DF = pd.DataFrame()

def Info_Extract(TickName):    
    global INF_DF
    try:
        tick = yf.Ticker(TickName) 
        INF = tick.info
        INF_DF = INF_DF.append(INF, ignore_index=True) 
         
        print("finished Info:",TickName)
        
    except Exception:
        print("Error with Info:",TickName)
        pass
    
    return()

# Download all
# for company in TICKER_ALL:
#    Info_Extract(company)


# download partial
# for company in TICKER_ALL_1:
#    Info_Extract(company)
# INF_DF_1 = INF_DF

# for company in TICKER_ALL_2:
#    Info_Extract(company)
# INF_DF_2 = INF_DF

# for company in TICKER_ALL_3:
#    Info_Extract(company)
# INF_DF_3 = INF_DF

# for company in TICKER_ALL_4:
#    Info_Extract(company)
# INF_DF_4 = INF_DF

# INF_DF_Master = pd.concat([INF_DF_1, INF_DF_2, INF_DF_3, INF_DF_4], ignore_index=True)

# for company in TICKER_ALL:
    # Info_Extract(company)
# INF_DF.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_INFO.pkl")

# INF_DF_1.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_INFO_1.pkl")
# INF_DF_2.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_INFO_2.pkl")
# INF_DF_3.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_INFO_3.pkl")
# INF_DF_4.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_INFO_4.pkl")


In [ ]:
#Download part 2: financial data

FINANCIALS_DF = pd.DataFrame()

def Financials_Extract(TickName):
    global FINANCIALS_DF
    try:
        tick = yf.Ticker(TickName) 
        
        #income statement
        FIN_OG = tick.financials
        FIN_OG2 = FIN_OG.reset_index()
        FIN_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        FIN_OG3=FIN_OG2.melt(id_vars=["Metric"],var_name="Date",value_name="Value")
        FIN_OG3['TickName']=TickName
        FIN_OG3['Financial Data Type']="Income Statement"
        
        FINANCIALS_DF = FINANCIALS_DF.append(FIN_OG3, ignore_index=True)
         
        #balance sheet
        BS_OG = tick.balance_sheet
        BS_OG2 = BS_OG.reset_index()
        BS_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        BS_OG3=BS_OG2.melt(id_vars=["Metric"],var_name="Date",value_name="Value")
        BS_OG3['TickName']=TickName
        BS_OG3['Financial Data Type']="Balance Sheet"
        
        FINANCIALS_DF = FINANCIALS_DF.append(BS_OG3, ignore_index=True)
        
        #cashflow
        CF_OG = tick.cashflow
        CF_OG2 = CF_OG.reset_index()
        CF_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        CF_OG3=CF_OG2.melt(id_vars=["Metric"],var_name="Date",value_name="Value")
        CF_OG3['TickName']=TickName
        CF_OG3['Financial Data Type']="Cashflow"
        
        FINANCIALS_DF = FINANCIALS_DF.append(CF_OG3, ignore_index=True)
        
            
        print("finished Financial:",TickName)
        
    except Exception:
        print("Error with Financial:",TickName)
        pass

    return()

# download all at once
# for company in TICKER_ALL:
#    Financials_Extract(company)


#  download partial
# for company in TICKER_ALL_1:
#    Financials_Extract(company)
# FINANCIALS_DF_1 = FINANCIALS_DF
# FINANCIALS_DF_1.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_FIN_1.pkl")

# for company in TICKER_ALL_2:
#    Financials_Extract(company)
# FINANCIALS_DF_2 = FINANCIALS_DF
# FINANCIALS_DF_2.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_FIN_2.pkl")

# for company in TICKER_ALL_3:
#    Financials_Extract(company)
# FINANCIALS_DF_3 = FINANCIALS_DF
# FINANCIALS_DF_3.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_FIN_3.pkl")

for company in TICKER_ALL_4:
   Financials_Extract(company)
FINANCIALS_DF_4 = FINANCIALS_DF
FINANCIALS_DF_4.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_FIN_4.pkl")

# INF_DF_Master = pd.concat([FINANCIALS_DF_1, FINANCIALS_DF_2, FINANCIALS_DF_3, FINANCIALS_DF_4], ignore_index=True)

# for company in TICKER_ALL:
    # Financials_Extract(company)
# FINANCIALS_DF.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_FIN.pkl")

# last download was 87mins

In [9]:
# note: 3 is partial
# FINANCIALS_DF_3 = FINANCIALS_DF
# FINANCIALS_DF_3.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_FIN_3.pkl")

FINANCIALS_DF_4 = FINANCIALS_DF
FINANCIALS_DF_4.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_FIN_4.pkl")

In [4]:
#Download part 3: Download historical share prices
HIST_PRICE_DF = pd.DataFrame()

def Prices_Extract(TickName):
    
    global HIST_PRICE_DF

    try:
        tick = yf.Ticker(TickName) 
        
        hist_p = tick.history(period="12mo")
        hist_p2 = hist_p.reset_index()
        hist_p2.rename(columns = {'index':'Metric'}, inplace = True)
        
        hist_p2['TickName']=TickName

        HIST_PRICE_DF = HIST_PRICE_DF.append(hist_p2, ignore_index=True)
        
            
        print("finished Price:",TickName)
        
    except Exception:
        print("Error with Price:",TickName)
        pass

    return()


for company in TICKER_ALL:
    Prices_Extract(company)

HIST_PRICE_DF.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_Prices.pkl")
# last download time was 20 mins

finished Price: 14D.AX
finished Price: 1AD.AX
finished Price: 1AE.AX
finished Price: 1AG.AX
finished Price: 1CG.AX
finished Price: 1MC.AX
finished Price: 1ST.AX
finished Price: 29M.AX
finished Price: 2BE.AX
finished Price: 360.AX
finished Price: 3DA.AX
finished Price: 3DP.AX
finished Price: 3MF.AX
finished Price: 3PL.AX
finished Price: 4DS.AX
finished Price: 4DX.AX
finished Price: 5EA.AX
finished Price: 5GG.AX
finished Price: 88E.AX
finished Price: 8CO.AX
finished Price: 8IH.AX
finished Price: 8VI.AX
finished Price: 92E.AX
finished Price: A11.AX
finished Price: A1G.AX
finished Price: A1M.AX
finished Price: A2B.AX
finished Price: A2M.AX
finished Price: A3D.AX
finished Price: A4N.AX
finished Price: A8G.AX
finished Price: AAC.AX
finished Price: AAJ.AX
finished Price: AAP.AX
finished Price: AAR.AX
finished Price: AAU.AX
finished Price: ABA.AX
finished Price: ABB.AX
finished Price: ABC.AX
finished Price: ABE.AX
finished Price: ABP.AX
finished Price: ABV.AX
finished Price: ABX.AX
finished Pr

In [5]:
#Part 4: historical dividend information

#DIV = tick.dividends  


HIST_DIV_DF = pd.DataFrame()

def Dividend_Extract(TickName):
    global HIST_DIV_DF
    try:
        tick = yf.Ticker(TickName) 
        
        #income statement
        DIV_OG = tick.dividends
        DIV_OG2 = DIV_OG.reset_index()
        DIV_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        DIV_OG2['TickName']=TickName
        
        HIST_DIV_DF = HIST_DIV_DF.append(DIV_OG2, ignore_index=True)
        
            
        print("finished Dividend",TickName)
        
    except Exception:
        print("Error with Dividend",TickName)

        pass

    return()

for company in TICKER_ALL:
   Dividend_Extract(company)

HIST_DIV_DF.to_pickle("/Users/joezhou/Downloads/Investment/1 data/R_ALL_DIV.pkl")

# last download time was 19 mins

finished Dividend 14D.AX
finished Dividend 1AD.AX
finished Dividend 1AE.AX
finished Dividend 1AG.AX
- 1CG.AX: 1d data not available for startTime=-2208988800 and endTime=1672443153. Only 100 years worth of day granularity data are allowed to be fetched per request.
Error with Dividend 1CG.AX
finished Dividend 1MC.AX
finished Dividend 1ST.AX
finished Dividend 29M.AX
finished Dividend 2BE.AX
finished Dividend 360.AX
finished Dividend 3DA.AX
finished Dividend 3DP.AX
finished Dividend 3MF.AX
finished Dividend 3PL.AX
finished Dividend 4DS.AX
finished Dividend 4DX.AX
finished Dividend 5EA.AX
finished Dividend 5GG.AX
finished Dividend 88E.AX
finished Dividend 8CO.AX
finished Dividend 8IH.AX
finished Dividend 8VI.AX
finished Dividend 92E.AX
finished Dividend A11.AX
finished Dividend A1G.AX
finished Dividend A1M.AX
finished Dividend A2B.AX
finished Dividend A2M.AX
finished Dividend A3D.AX
finished Dividend A4N.AX
finished Dividend A8G.AX
finished Dividend AAC.AX
finished Dividend AAJ.AX
finishe

In [ ]:
#Inject data into SQL lite database or some kind of clouud database

In [ ]:
#Insert data
#DF_SHORT.to_sql('test', db_sql, if_exists='replace')

#pd.read_sql('select * from test', db_sql)

In [ ]:
# pip install yfinance==0.1.62

# ticker = yf.Ticker('AAPL')
# aapl_df = ticker.history(period="1y")
# aapl_df['Close'].plot(title="APPLE's stock price")

# aapl_df = yf.download('AAPL', 
                    #   start='2022-01-01', 
                    #   end='2022-03-16', 
                    #   progress=False,
# )
# aapl_df.head()